# *DOWNLOAD PROXIES FOR WEBSCRAPING*

In [1]:
# Import the Scraper functionality
import Scraper

In [4]:
# Define the routine for obtaining proxies from the *https://free-proxy-list.net/*
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup

# The link that we will scrape.
link = 'https://free-proxy-list.net/'

# Define the scraper routine that you will use at the link
# RETURN: a pandas dataframe containing the data you want to collect.
def scrape_routine_proxies(browser):
    
    # Select the option for the most numerous proxies
    select = Select(browser.find_element_by_xpath('/html/body/section[1]/div/div[2]/div/div[1]/div[1]/div/label/select'))
    elem = select.select_by_visible_text('80')

    # Read the table and return a dataframe
    soup = BeautifulSoup(browser.page_source,'html.parser')
    table = soup.find('table', {'id' :"proxylisttable"})
    headers = [h.text for h in table.find_all('th',{'aria-controls':"proxylisttable"})]
    
    rows = table.find_all('tr', {'class': "odd"}) + table.find_all('tr', {'class': "even"})
    data = zip(*[[r.text for r in R.find_all('td')] for R in rows])
    
    return pd.DataFrame({a:b for (a,b) in zip(headers,data)})

In [6]:
import pandas as pd

# Scrape the web proxies, then close the scraper object
scr = Scraper.Scraper(isHeadless=True)
df = scr.scrape_link(link,scrape_routine_proxies)
scr.close()

# properly format the columns for postgresql (i.e. lower case headers, no spaces.)
df.columns = [header.lower().replace(" ","_") for header in df.columns]

In [7]:
# CREATE A POSTGRES TABLE TO STORE THESE PROXIES FOR USE LATER
from sqlalchemy import create_engine, Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.engine.url import URL
import db_configuration

DB_CONFIG_DICT = db_configuration.DB_CONFIG_DICT
DB_CONFIG_DICT['database'] = 'wills_db'

DB_CONN_FORMAT = "postgresql://{user}:{password}@{host}:{port}/{database}"
DB_CONN_URI_DEFAULT = (DB_CONN_FORMAT.format(**DB_CONFIG_DICT))

# Create a function for a quick connection to the database
def db_connect():
    return create_engine(DB_CONN_URI_DEFAULT)

In [8]:
conn = db_connect()
conn.execute("DROP TABLE IF EXISTS proxies;")
df.to_sql('proxies',conn,index=None)

In [9]:
conn = db_connect()

# Get the first ten results
conn.execute("SELECT * FROM proxies;").fetchall()[0:10]

[('168.228.107.180', '8080', 'BR', 'Brazil', 'transparent', 'no', 'no', '20 seconds ago'),
 ('159.203.20.110', '8080', 'CA', 'Canada', 'anonymous', 'no', 'no', '22 seconds ago'),
 ('104.248.115.226', '8080', 'US', 'United States', 'anonymous', 'no', 'no', '22 seconds ago'),
 ('157.230.149.54', '80', 'US', 'United States', 'anonymous', 'no', 'no', '22 seconds ago'),
 ('157.230.157.60', '8080', 'US', 'United States', 'anonymous', 'no', 'no', '22 seconds ago'),
 ('206.81.11.75', '80', 'US', 'United States', 'anonymous', 'no', 'no', '22 seconds ago'),
 ('157.230.140.12', '8080', 'US', 'United States', 'anonymous', 'no', 'no', '22 seconds ago'),
 ('35.198.28.187', '8080', 'US', 'United States', 'elite proxy', 'no', 'no', '22 seconds ago'),
 ('134.209.231.163', '8080', 'DE', 'Germany', 'anonymous', 'no', 'no', '22 seconds ago'),
 ('157.230.210.133', '8080', 'US', 'United States', 'anonymous', 'no', 'no', '22 seconds ago')]